In [9]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import whisper
from pydub import AudioSegment
from transformers import RobertaTokenizer
import tensorflow as tf
from transformers import TFRobertaForSequenceClassification
from Pipeline_functions import convert_mov_to_mp3, predicting, transcription, post_transcription, map_emotions_and_find_common, mp3_to_segments

# Step 1: Converting the MOV files to MP3

In [2]:
# Directory where the original videofiles are located
input_directory = r"C:\Users\daraz\Desktop\BUas\Year 2 Block C\Data\VIDEO DATA\Afleveringen\Afleveringen"

# Converting the mov files to mp3 files
convert_mov_to_mp3(input_directory)

Successfully converted C:\Users\daraz\Desktop\BUas\Year 2 Block C\Data\VIDEO DATA\Afleveringen\Afleveringen\ER22_AFL01_MXF.mov to C:\Users\daraz\Desktop\BUas\Year 2 Block C\Tasks\Pipeline\mp3s\ER22_AFL1_MXF.mp3
Successfully converted C:\Users\daraz\Desktop\BUas\Year 2 Block C\Data\VIDEO DATA\Afleveringen\Afleveringen\ER22_AFL02_MXF.mov to C:\Users\daraz\Desktop\BUas\Year 2 Block C\Tasks\Pipeline\mp3s\ER22_AFL2_MXF.mp3
Successfully converted C:\Users\daraz\Desktop\BUas\Year 2 Block C\Data\VIDEO DATA\Afleveringen\Afleveringen\ER22_AFL03_MXF.mov to C:\Users\daraz\Desktop\BUas\Year 2 Block C\Tasks\Pipeline\mp3s\ER22_AFL3_MXF.mp3
Successfully converted C:\Users\daraz\Desktop\BUas\Year 2 Block C\Data\VIDEO DATA\Afleveringen\Afleveringen\ER22_AFL04_MXF.mov to C:\Users\daraz\Desktop\BUas\Year 2 Block C\Tasks\Pipeline\mp3s\ER22_AFL4_MXF.mp3
Successfully converted C:\Users\daraz\Desktop\BUas\Year 2 Block C\Data\VIDEO DATA\Afleveringen\Afleveringen\ER22_AFL05_MXF.mov to C:\Users\daraz\Desktop\BUa

# Step 2: Slicing the mp3s into smaller segments

In [3]:
for i in range(1, 18):
        # Loading in the audio file and the structure file
        mp3 = f'C:/Users/daraz/Desktop/BUas/Year 2 Block C/Tasks/Pipeline/mp3s/ER22_AFL{i}_MXF.mp3'
        structure = pd.read_csv(f"C:/Users/daraz/Desktop/BUas/Year 2 Block C/Data/VIDEO DATA/OneDrive_2024-03-29/Data Banijay Expeditie Robinson/ER22_AFL{i}_structure.csv")

        audio_full = AudioSegment.from_mp3(mp3)

        # Looping through the structure file to extract the audio segments
        for index, row in structure.iterrows():

                # Extracting the audio segment
                segment = mp3_to_segments(audio_full, row)

                # Saving the audio segment
                segment.export(f"C:/Users/daraz/Desktop/BUas/Year 2 Block C/Tasks/Pipeline/segments/ER22_AFL{i}_segment_{index}.mp3", format="mp3")

                print(f"ER22_AFL{i}_segment_{index}.mp3 has been saved")

ER22_AFL1_segment_0.mp3 has been saved
ER22_AFL1_segment_1.mp3 has been saved
ER22_AFL1_segment_2.mp3 has been saved
ER22_AFL1_segment_3.mp3 has been saved
ER22_AFL1_segment_4.mp3 has been saved
ER22_AFL1_segment_5.mp3 has been saved
ER22_AFL1_segment_6.mp3 has been saved
ER22_AFL1_segment_7.mp3 has been saved
ER22_AFL1_segment_8.mp3 has been saved
ER22_AFL1_segment_9.mp3 has been saved
ER22_AFL1_segment_10.mp3 has been saved
ER22_AFL1_segment_11.mp3 has been saved
ER22_AFL1_segment_12.mp3 has been saved
ER22_AFL1_segment_13.mp3 has been saved
ER22_AFL1_segment_14.mp3 has been saved
ER22_AFL1_segment_15.mp3 has been saved
ER22_AFL1_segment_16.mp3 has been saved
ER22_AFL1_segment_17.mp3 has been saved
ER22_AFL1_segment_18.mp3 has been saved
ER22_AFL1_segment_19.mp3 has been saved
ER22_AFL2_segment_0.mp3 has been saved
ER22_AFL2_segment_1.mp3 has been saved
ER22_AFL2_segment_2.mp3 has been saved
ER22_AFL2_segment_3.mp3 has been saved
ER22_AFL2_segment_4.mp3 has been saved
ER22_AFL2_segme

# Step 3: Transcribing the MP3 files and saving them as Pandas DataFrames

In [8]:
# Directory where the mp3 files are located
input_directory = r"C:\Users\daraz\Desktop\BUas\Year 2 Block C\Tasks\Pipeline\segments"

# Loading the model
model = whisper.load_model("tiny")

# Transcribing the mp3 files
transcription(input_directory, model)

Translated ER22_AFL1_segment_0.mp3
Translated ER22_AFL1_segment_1.mp3
Translated ER22_AFL1_segment_2.mp3
Translated ER22_AFL1_segment_3.mp3
Translated ER22_AFL1_segment_4.mp3
Translated ER22_AFL1_segment_5.mp3
Translated ER22_AFL1_segment_6.mp3
Translated ER22_AFL1_segment_7.mp3
Translated ER22_AFL1_segment_8.mp3
Translated ER22_AFL1_segment_9.mp3
Translated ER22_AFL1_segment_10.mp3
Translated ER22_AFL1_segment_11.mp3
Translated ER22_AFL1_segment_12.mp3
Translated ER22_AFL1_segment_13.mp3
Translated ER22_AFL1_segment_14.mp3
Translated ER22_AFL1_segment_15.mp3
Translated ER22_AFL1_segment_16.mp3
Translated ER22_AFL1_segment_17.mp3
Translated ER22_AFL1_segment_18.mp3
Translated ER22_AFL1_segment_19.mp3
Translated ER22_AFL2_segment_0.mp3
Saved ep1_translations.csv
Translated ER22_AFL2_segment_1.mp3
Translated ER22_AFL2_segment_2.mp3
Translated ER22_AFL2_segment_3.mp3
Translated ER22_AFL2_segment_4.mp3
Translated ER22_AFL2_segment_5.mp3
Translated ER22_AFL2_segment_6.mp3
Translated ER22_AF

# Step 4: Combining the transcriptions with the annotated emotions for each episode and turning them into a single DataFrame

In [33]:
# Creating a dataframe that will store the merged datasets
bigmerged = pd.DataFrame()

# Merging the structure and transcript dataframes for each episode
for i in range(1, 18):

    # Reading in the transcript and structure dataframes
    transcript = pd.read_csv(f"C:/Users/daraz/Desktop/BUas/Year 2 Block C/Tasks/Pipeline/transcription_dfs/ep{i}_translations.csv")
    structure = pd.read_csv(f"C:/Users/daraz/Desktop/BUas/Year 2 Block C/Data/VIDEO DATA/OneDrive_2024-03-29/Data Banijay Expeditie Robinson/ER22_AFL{i}_structure.csv")

    # large_v3 model transcripts
    transcript = pd.read_csv(f"C:/Users/daraz/Desktop/BUas/Year 2 Block C/Data/VIDEO DATA/Transcriptions/ep{i}_translations.csv")

    # Merging
    merged = post_transcription(structure, transcript, bigmerged)

    # Concatenating the merged dataframe to the bigmerged dataframe
    bigmerged = pd.concat([bigmerged, merged])

# Dropping the "row_number" column
bigmerged = bigmerged.drop(columns=['row_number'])

# Applying the function to the "Emotions" column
bigmerged['Most_Common_Basic_Emotion'] = bigmerged['Emotions'].apply(map_emotions_and_find_common)

# Renaming 'Emotions' column to 'Labeled_Emotion' and Basic_Emotion column to 'Predicted_Emotion'
bigmerged = bigmerged.rename(columns={'Emotions': 'Labeled_Emotion', 'Most_Common_Basic_Emotion': 'Labeled_emotion_fromsix', 'Basic_Emotion': 'Predicted_Emotion_All'})

bigmerged

,Episode name,Start Time (seconds),End Time (seconds),Labeled_Emotion,Translation,Labeled_emotion_fromsix
0,1,124,350,"Joy, Surprise, Anticipation",But something is wrong. We're on a boat on ou...,surprise
1,1,1031,1078,Joy,Still no cauldron found. They are lying here....,happiness
2,1,1111,1178,Disgust,Two tubes are needed. Important tubes. I need...,disgust
3,1,1210,1256,"Joy, Disappointment",At some point I was digging with two fingers ...,happiness
4,1,1292,1353,Joy,"The first one is found, then the second one. ...",happiness
5,1,1981,2290,"Fear, Amusement, Optimism, Anticipation, Disap...","3, 2, 1, go! As team Orange we immediately ha...",happiness
6,1,2323,2951,"Joy, Annoyance, Excitement, Optimism, Pride, A...","For the green team, Team Purple is neck and n...",happiness
7,1,2975,3067,"Joy, Optimism, Pride","One down for team green, one down for team or...",happiness
8,1,3067,3233,"Joy, Disappointment","First team green, with the win. First proof i...",happiness
9,1,3233,3265,Joy,The teams leave to an uninhabited island. Wit...,happiness


# Step 5: Predicting the emotion of each segment

In [34]:
sentences = bigmerged['Translation']

In [35]:
# Initialize the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Maximum sequence length
MAX_LENGTH = 512

# Tokenize sentences to get input IDs and attention masks
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LENGTH,
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,  # Return attention masks
                        return_tensors = 'tf',  # Return TensorFlow tensors
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert lists of tensors to a single tensor
input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)

In [36]:
# Loading the model
model_directory = r"C:\Users\daraz\Desktop\BUas\Year 2 Block C\Tasks\Pipeline\model"

# Load the fine-tuned model
model = TFRobertaForSequenceClassification.from_pretrained(model_directory)

# Make predictions
predictions = model.predict({'input_ids': input_ids, 'attention_mask': attention_masks})

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at C:\Users\daraz\Desktop\BUas\Year 2 Block C\Tasks\Pipeline\model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


14/14 [==============================] - 224s 15s/step


In [37]:
emotion_mapping = {
    1: "anger",
    4: "disgust",
    0: "fear",
    2: "happiness",
    5: "sadness",
    3: "surprise"
}

In [40]:
logits = predictions.logits

# Convert logits to probabilities
probabilities = tf.nn.softmax(logits, axis=-1).numpy()

# Option 2: Get the predicted class (with the highest probability)
predicted_classes = np.argmax(probabilities, axis=-1)

bigmerged['Predicted_Emotion'] = predicted_classes

# Applying the mapping to convert numerical predictions to emotion labels
bigmerged['Predicted_Emotion_Label'] = bigmerged['Predicted_Emotion'].map(emotion_mapping)


bigmerged['Predicted_Emotion'] = predicted_classes

# Making all rows visible
pd.set_option('display.max_rows', None)

bigmerged

,Episode name,Start Time (seconds),End Time (seconds),Labeled_Emotion,Translation,Labeled_emotion_fromsix,Predicted_Emotion,Predicted_Emotion_Label
0,1,124,350,"Joy, Surprise, Anticipation",But something is wrong. We're on a boat on ou...,surprise,2,happiness
1,1,1031,1078,Joy,Still no cauldron found. They are lying here....,happiness,2,happiness
2,1,1111,1178,Disgust,Two tubes are needed. Important tubes. I need...,disgust,2,happiness
3,1,1210,1256,"Joy, Disappointment",At some point I was digging with two fingers ...,happiness,2,happiness
4,1,1292,1353,Joy,"The first one is found, then the second one. ...",happiness,2,happiness
5,1,1981,2290,"Fear, Amusement, Optimism, Anticipation, Disap...","3, 2, 1, go! As team Orange we immediately ha...",happiness,2,happiness
6,1,2323,2951,"Joy, Annoyance, Excitement, Optimism, Pride, A...","For the green team, Team Purple is neck and n...",happiness,2,happiness
7,1,2975,3067,"Joy, Optimism, Pride","One down for team green, one down for team or...",happiness,2,happiness
8,1,3067,3233,"Joy, Disappointment","First team green, with the win. First proof i...",happiness,2,happiness
9,1,3233,3265,Joy,The teams leave to an uninhabited island. Wit...,happiness,3,surprise


In [ ]:
bigmerged['Predicted_Emotion'].value_counts()

Predicted_Emotion
2    321
3     82
1     31
5      7
0      1
Name: count, dtype: int64

In [ ]:
bigmerged['Labeled_emotion_fromsix'].value_counts()

Labeled_emotion_fromsix
happiness    268
surprise      81
sadness       43
anger         36
fear           8
disgust        6
Name: count, dtype: int64